In [ ]:
#I HAVE ALL MY DATA AND MODELS STORED IN MY GOOGLE DRIVE SO WE CONNECT TO THAT 
from google.colab import drive
drive.mount('gdrive', force_remount=True)
root_dir = '/content/gdrive/'

Mounted at gdrive


In [ ]:
#IMPORT THE MODULES WE NEED 
%tensorflow_version 1.x
from tensorflow import keras
from tensorflow.keras import optimizers
!pip install keras


TensorFlow 1.x selected.


In [ ]:
#MORE REQUIRED MODULES
!pip install transformers
!pip install pytorch_lightning
!pip install simpletransformers
  
import pandas as pd
from simpletransformers.t5 import T5Model
from pprint import pprint
import logging
from sklearn.model_selection import train_test_split

In [ ]:
#WE ARE GOING TO TRAIN THE T5 MODEL ON THE TRAINING DATASET PROVIDED IN THIS ASSIGNMENT
columns = ["Input", "Output"]
df = pd.read_csv('/content/gdrive/MyDrive/public_dat/train/train.csv', header=None, names=columns)


#WE NEED TO SPLIT THE DATASET 90/10 TO TEST VALIDATION 

train, test = train_test_split(df, test_size=0.1)

#SETUP THE DATA IN THE REQUIRED MODEL FORMAT
train["prefix"] = "Generate Paraphrase for this line"
train = train.rename(
  columns={"Input": "input_text", "Output": "target_text"})
  
#VALIDATION DATA
test["prefix"] = "Generate Paraphrase for this line"
test = test.rename(
  columns={"Input": "input_text", "Output": "target_text"})



In [ ]:
#THESE ARE THE MODEL PARAMETERS WE ARE GOING TO USE
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 32,
    "eval_batch_size": 64,
    "num_train_epochs": 10,
    "num_beams": None,
    "do_sample": True,
    "max_length": 20,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": True,
    "evaluate_during_training_verbose": True,
    "num_return_sequences": 10
}



In [ ]:
#RE-TRAIN THE MODEL ON MY DATA
#THIS IS ESENTIALLY TRANSFER LEARNING 
model = T5Model("t5","t5-base", args=model_args, use_cuda=False)
model.train_model(train, eval_data=test)

  0%|          | 0/1006 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Using Adafactor for T5


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

(320,
 {'eval_loss': [0.7857931852340698,
   0.7942772209644318,
   0.8657564222812653,
   0.955542802810669,
   1.1070712804794312,
   1.1550079584121704,
   1.2225521206855774,
   1.2550709247589111,
   1.2915058732032776,
   1.3521144390106201],
  'global_step': [32, 64, 96, 128, 160, 192, 224, 256, 288, 320],
  'train_loss': [0.5879908204078674,
   0.5323508381843567,
   0.49824628233909607,
   0.27242231369018555,
   0.16890300810337067,
   0.09565546363592148,
   0.14409440755844116,
   0.11274038255214691,
   0.0614159069955349,
   0.057361193001270294]})

#################################################################################

In [ ]:
#COPY THE MODEL TO GOOGLE DRIVE SO IT DOES NOT NEED TO BE RUN AGAIN 
!cp -r /content/outputs/ /content/gdrive/'MyDrive'/newest_model/


In [ ]:
#TAKE THE BEST MODEL FROM THE PREVIOUSLY SAVED MODEL
#ONLY NEED TO RUN THIS WHEN WORKING WITH THE PROJECT 
newmodel = T5Model("t5", "/content/gdrive/MyDrive/newest_model/best_model/", args=model_args, use_cuda=False)


In [ ]:
#NOW LETS USE THIS NEWLY TRAINED T5 MODEL WHICH HAS BEEN TUNED TO THE DATASET TO 
#MAKE SOME PREDICTIONS ON THE TEST DATA

columns = ["Input"]
df = pd.read_csv('/content/gdrive/MyDrive/public_dat/test/eval.csv', header=None, names=columns)
df.head()


#lets take the first sample
df["prefix"] = "Generate Paraphrase for this line"
df = df.rename(
  columns={"Input": "input_text"})

test_list = df["input_text"].tolist()


df.head()

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(df["prefix"].tolist(), df["input_text"].tolist())
]

preds = newmodel.predict(to_predict)



Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
#HERE IS WHERE WE START WITH OUR SIAMESE RNN IMPLEMENTATION 
#NOTE THAT THE SIAMESE RNN HERE IS TAKEN FROM THE https://github.com/MahmoudWahdan/Siamese-Sentence-Similarity IMPLEMENTATION

In [ ]:
#CLONE THE REPOSITORY SO WE CAN USE THE MODEL
#THE NEWLY TRINED MODEL WILL RETURN 10 POSSIBLE SENTENCES SO WE USE THIS 
#SIAMESE RNN TO SAY WHICH SENTENCE IS MOST SIMILAR AND THEREFORE THE BEST 
!git clone https://github.com/MahmoudWahdan/Siamese-Sentence-Similarity.git

Cloning into 'Siamese-Sentence-Similarity'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75
Unpacking objects: 100% (75/75), done.


In [ ]:
#GO TO THE MODEL DIRECTORY
%cd /content/Siamese-Sentence-Similarity/

/content/Siamese-Sentence-Similarity


In [ ]:
#IMPORT EVERYTHING NEEDED FROM THE IMPORTED GITHUB ACCOUNT
import tokenizer as tok
import word_embeddings as word
import vectorizer as vect
import siamese as siam 
import data_reader as dr
import utils as utils 

import importlib
importlib.reload(siam)
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#WE USE THE WORD2VEC GOOGLE WORD EMBEDDINGS
#TRAIN ON THE SICK DATASRT
#TRAIN FOR 50 EPOCHS
word_embeddings_file_path = '/content/gdrive/MyDrive/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'
train_df, val_df, test_df = dr.read_SICK_data('/content/gdrive/MyDrive/SICK.txt')
epochs = 50
save_path = '/content/gdrive/MyDrive/siamese_model/'
use_cudnn_lstm = True 

#CREATE A TOEKNIZER AND VECTORIZER USING THE WORD2VEC WORD EMBEDDINGS
word_embeddings = word.WordEmbeddings(word_embeddings_file_path)
tokenizer = tok.Tokenizer()
vectorizer = vect.Vectorizer(word_embeddings, tokenizer)

#READ IN THE TRAINING DATA
ids, train_a_vectors, train_b_vectors, train_gold = vectorizer.vectorize_df(train_df)
train_max_a_length = len(max(train_a_vectors, key=len))
train_max_b_length = len(max(train_b_vectors, key=len))
max_len = max([train_max_a_length, train_max_b_length])

#PADDING STEP
train_a_vectors = utils.pad_tensor(train_a_vectors, max_len)
train_b_vectors = utils.pad_tensor(train_b_vectors, max_len)


#VALIDATION DATASET
ids, val_a_vectors, val_b_vectors, val_gold = vectorizer.vectorize_df(val_df)
val_max_a_length = len(max(val_a_vectors, key=len))
val_max_b_length = len(max(val_b_vectors, key=len))
max_len = max([val_max_a_length, val_max_b_length])

#PADDING STEP
val_a_vectors = utils.pad_tensor(val_a_vectors, max_len)
val_b_vectors = utils.pad_tensor(val_b_vectors, max_len)

#LOAD THE MODEL
siamese = siam.SiameseModel(use_cudnn_lstm)

#SETUP THE VALIDATION DATA
validation_data = ([val_a_vectors, val_b_vectors], val_gold)

#TRAIN THE MODEL
siamese.fit(train_a_vectors, train_b_vectors, train_gold, validation_data, epochs=epochs)

#SAVE THE MODEL
if save_path is not None:
    siamese.save(model_folder=save_path)



In [ ]:
#AGAIN CREATE THESE OBJECTS FOR USING THE MODEL TO SELECT THE BEST SENTENCE FROM THE 10 GENERATED BY THE M5 MODEL
word_embeddings = word.WordEmbeddings('/content/gdrive/MyDrive/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin')
tokenizer = tok.Tokenizer()
vectorizer = vect.Vectorizer(word_embeddings, tokenizer)

#NOW WE LOAD THE SAVED MODE THAT WE SAVED PREVIOUSLY
siamese = siam.SiameseModel(False)
siamese.load('/content/gdrive/MyDrive/siamese_model/')

#NOW WE NEED TO USE THIS MODEL TO MAKE PREDICTIONS


Number of Tokens: 3000000
Dimension of a word vector: 300
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 300)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 300)]  0                                            
__________________________________________________________________________________________________
lstm_layer (LSTM)               (None, 50)           70200       input_1[0][0]                    
                                                                 input_2[0][0]                    
______________________________________________________

In [ ]:
#THERE IS ABOUT 15/100 CASES WHERE THE M5 MODEL MAKE NO NEW PREDICTIONS FOR SENTENCES
#THIS MEANS WE NEED TO USE ANOTHER APPROACH FOR THESE CASES 
#THE APPROACH I HAVE CHOSEN IS THE GENERATE SYNONYMS OF WORDS USING WORDNET SYNSETS
#THIS WILL GIVE 10 PROPOSED SENTENCES 
#I WILL THEN USE THE SIAMESE RNN TO SEE WHICH IS BEST 

#HERE WE CREATE A FUNCTION THAT
#TAKES IN A SENTENCE
#LOOPS THROUGH THE WORDS 
#USES WORDNET SYSSETS TO GENERATE NEW WORDS FOR WORDS THAT ARE NOT STOPWORDS 
#THEN ALTERS THE SENTENCE 

nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

#HERE IS THE FUNCTION
def paraphrase_sentence(sentence):

  #SETUP THE ACTUAL SENTENCE SO IT CAN BE PSSED INTO THE SIAMESE RNN MODEL 
  actual = vectorizer.vectorize_sentence(sentence)
  actual = np.array(actual)
  actual = np.expand_dims(actual, axis=0)

  #SOME INITIALIZATION 
  possible_new_sents = []
  sent_cpy = sentence
  sent_cpy = word_tokenize(sent_cpy)
  word_synonyms_pairs = {}
  empty = []

  #LOOP THROUGH THE WORDS IN THE SENTENCE WE WANT TO GENERATE NEW WORDS FOR
  for word in sent_cpy:
    word_synonyms = []
    #IF THE WORD IS NOT A STOPWORD
    if word.lower() not in stop_words: 
      #LOOP THROUGH THE SYSNETS FOR THAT WORD
      for synset in wordnet.synsets(word):
        #LOOP THROUGH THE LEMMAS
        for l in synset.lemmas():
          #IF THE NEW WORD IS NOT THE SAME AS THE ORIGINAL WORD
          if l.name().lower() != word.lower():
            word_synonyms.append(l.name())
      
      #IF ITS NOT EMPTY WE ADD THE WORD AND A LIST OF POSSABILITIES TO A DICTIONARY 
      if word_synonyms != empty:
        word_synonyms_pairs[word] = word_synonyms


  #NOW WE NEED TO LOOP THROUGH DICTIONARIES AND CREATE A NUMBER OF SENTENCES

  #SOME INITIAILIZATION
  possible_sentences = []
  num_words = len(word_synonyms_pairs)
  num_new_sents = 10
  
  #CREATE 10 COPIES OF THE SENTENCE IN A LIST (WE WILL ALTER THESE WITH THE OPTIONS)
  sentences = [None] * num_new_sents

  word_num = 0

  #LOOP THROUGH THE 10 (COPIES OF ORIGINAL SENTENCE) SENTENCES WE WILL MODIFY TO CREATE NEW ONES
  for i in range(num_new_sents):
    #PUT THE ORIGINAL SENTENCE IN IT AT EACH LOCATION
    sentences[i] = sentence

  #LOOP THROUGH ALL THE WORDS/KEYS IN THE DICTIONARY
  #THESE ARE THE WORDS WE ARE LOOKING TO CHANGE 
  for word in word_synonyms_pairs:

    counter = 0
    word_num = word_num + 1

    #ALL THE POSSIBLE NEW WORDS (SYONYMS) FOR THAT SINGLE WORD
    all_words = word_synonyms_pairs[word]
    top_words = all_words[:10]

    #IF THERE IS LESS THAN 10 POSSIBLE WORDS FOR THAT WORD 
    #FILL UP THE LIST BY ITERATING OVER THE POSSIBLE ONES
    if len(top_words) < 10:
      x = len(top_words)
      index = x
      val = 0 

      #HERE IS WHERE WE PASTE THE POSSIBLE WORDS IN TO THE SPACES TO FILL UP THE LIST
      while index < 10:
        index = index + 1
        top_words.append(top_words[val])
        val = val + 1

    #LOOP THROUGH ALL THE POSSIBLE WORDS
    for new_words in top_words: 

      #REPLACE THE ACTUAL WORD WITH THE NEW POSSIBLE WORD
      sentences[counter] = sentences[counter].replace(word, new_words)
      counter = counter + 1


  #NOW WE HAVE A LIST OF ALL THE POSSIBLE SENTENCES WE NEED TO SELECT THE BEST
  #WE DO SO USING THE SIAMESE RNN AS THIS WILL SELECT THE MOST SILIAR SENTENCE IN CONTEXT

  #LOOP THROUGH POSSIBLE SENTENCES
  for k in range(len(sentences)):

    #SETUP THE NEW SENTENCE SO IT CAN BE PASSED INTO THE SIAMESE RNN AND COMPARED TO THE ORIGINAL
    para = vectorizer.vectorize_sentence(sentences[k])
    max_length = max([len(para), len(actual)])
    para = np.array(para)
    para = np.expand_dims(para, axis=0)

    #GET THE SIMILARITY PREDICTION
    y = siamese.predict(para, actual)
    #APPEND THE RESULT TO AN ARRAY 
    possible_sentences.append(y)
  
  #GET THE INDEX OF THE SENTENCE THAT MATCHES THE MEANING OF THE ORIGINAL BEST 
  max_index = possible_sentences.index(max(possible_sentences))

  #RETURN THE BEST SENTENCE
  return sentences[max_index]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#HERE IS WHERE I LOOP THROUGH EACH OF THE SENTENCES AND CREATE PARAPHRASES

#LIST FOR THE RESULTS

results = []
import numpy as np 

#TOTALS OF THE TYPES OF WORK I DO
total_siamese = 0
total_all_same = 0
one_new = 0

#LOOP THROUGH THE PREDICTIONS
for i in range(len(preds)):

  all_same = False
  possible_paraphrase = []


  for j in range(len(preds[i])):

    #IF THE PREDICTION IS THE SAME AS THE ORIGINAL SENTENCE DO NOT ADD IT AS A PARAPHRASE
    if(test_list[i] == preds[i][j]):
      pass
    
    #IF ITS DIFFERENT ADD IT TO A LIST 
    else:
      possible_paraphrase.append(preds[i][j])

  #IF THE LIST OF POSSIBLE PARAPHRASES IS EMPTY THEN WE KNOW THAT THE PARAPHRASER DID NOT MAKE A NEW PREDICTION
  #THIS MEANS WE MUST DO SOMETHING ELSE
  if not possible_paraphrase:

    #WE CAN ON THE METHOD TO GENERATE A NEW PARAPHRASE USING THE WORDNET SYNONYS AND THE SIAMESE RNN
    res = paraphrase_sentence(test_list[i])

    #INCREMENT COUNTER FOR ANALYSIS 
    total_all_same = total_all_same + 1

    #APPEND THE RESULT
    results.append(res)

  #IF THERE IS ONLY ONE POSSIBLE GENERATED PARAPHRASE WHICH IS OFTEN THE EXACT SAME AS THE INPUT SENTENCE
  elif (len(possible_paraphrase) == 1):

    ##DO THE SAME AGAIN HERE USING WORDNET SYNONYMS AND THE SIAMESE RNN FUNCTION 
    one_new = one_new + 1
    res = paraphrase_sentence(test_list[i])
    results.append(res)

  else:

    total_siamese = total_siamese + 1

    #IN THIS CASE WE HAVE A NUMBER OF POSSIBLE SENTENCES AND WE JUST CHOOSE THE BEST ONE
    #USING THE SIAMESE RNN

    #SETUP THE ACTUAL SENTENCE SO IT CAN BE USED IN THE SIAMESE RNN
    actual = vectorizer.vectorize_sentence(test_list[i])
    actual = np.array(actual)
    actual = np.expand_dims(actual, axis=0)

    similarities = []

    #LOOP THROUGH THE POSSABILTIIES
    for k in range(len(possible_paraphrase)):

      #SETUP SAMPLE SENTENCE SO IT CAN BE USED IN SIAMESE RNN
      para = vectorizer.vectorize_sentence(possible_paraphrase[k])
      max_length = max([len(para), len(actual)])
      para = np.array(para)
      para = np.expand_dims(para, axis=0)

      #GET SIMILARITY
      y = siamese.predict(para, actual)
      similarities.append(y)
    
    #TAKE THE MOST SIMILAR
    index = similarities.index(max(similarities))
    results.append(possible_paraphrase[index])


print(total_siamese)
print(one_new)
print(total_all_same)


76
18
6


In [ ]:
#PRINTS OUT THE RESULTING SENTENCES

for i in range(len(results)):
  print(results[i])

In [ ]:
import csv

#HERE WE WRITE THE RESULTING DATA TO THE CSV FILE THAT WE ARE REQUIRED TO SUBMIT TO CODALAB 
with open('/content/eval.csv','w') as out:
    csv_out=csv.writer(out)
    for i in range(len(results)):
        resnew = (test_list[i],results[i])
        csv_out.writerow(resnew)